In [1]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime

device_name = sys.argv[1]  # Choose device from cmd line. Options: gpu or cpu
shape = (int(sys.argv[2]), int(sys.argv[2]))
if device_name == "gpu":
    device_name = "/gpu:0"
else:
    device_name = "/cpu:0"

with tf.device(device_name):
    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
    sum_operation = tf.reduce_sum(dot_operation)


startTime = datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
        result = session.run(sum_operation)
        print(result)

# It can be hard to see the results on the terminal with lots of output -- add some newlines to improve readability.
print("\n" * 5)
print("Shape:", shape, "Device:", device_name)
print("Time taken:", datetime.now() - startTime)

print("\n" * 5)

ValueError: invalid literal for int() with base 10: 'C:\\Users\\singh\\AppData\\Roaming\\jupyter\\runtime\\kernel-7884e5a0-81c8-4cfd-9bf6-f1c01eed6c45.json'

In [2]:
tf.Criti()

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [5]:
tf.config.experimental

<module 'tensorflow_core._api.v2.config.experimental' from 'C:\\Users\\singh\\Anaconda3\\lib\\site-packages\\tensorflow_core\\_api\\v2\\config\\experimental\\__init__.py'>

In [16]:
import tensorflow as tf
from tensorflow import keras

config = tf.config.experimental.list_physical_devices('GPU')
sess = tf.compat.v1.Session(config=config)

TypeError: config must be a tf.ConfigProto, but got <class 'list'>

In [20]:
# Allow memory growth for the GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

True

In [13]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [26]:
import tensorflow as tf
config = tf.config.experimental
sess = tf.compat.v1.Session()

In [30]:
sess.list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 3482480771015484474),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 3142752665, 1818553643708106786)]

In [31]:
sess.graph_def

versions {
  producer: 175
}

In [32]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [38]:
tf.config.experimental.list_physical_devices('GPU')
sess = tf.config.experimental.set_visible_devices(physical_devices[0:], 'GPU')

In [39]:
sess

In [44]:
a =config.get_visible_devices = '1'

In [45]:
a

'1'

In [48]:
tf.executing_eagerly()

True

In [49]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [50]:
visible_device_list

NameError: name 'visible_device_list' is not defined

In [51]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [52]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [95]:
tf.__version__

'2.1.0'

In [99]:
gpus = tf.config.experimental.list_physical_devices("GPU")

In [98]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [56]:
 tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [59]:
tf.config.experimental.set_memory_growth

True

In [60]:
tf.constant(1)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [64]:
tf.config.experimental.set_visible_devices([0], 'GPU')

ValueError: Unrecognized device: 0

In [88]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

In [89]:
tf.config.experimental.set_visible_devices(gpus[:], 'GPU')

In [90]:
visible_devices = tf.config.experimental.get_visible_devices

In [91]:
visible_devices

'1'

In [92]:
x = tf.Variable(1.0)

In [93]:
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [94]:
tf.cuda

AttributeError: module 'tensorflow' has no attribute 'cuda'

In [82]:
import argparse
import os
import time
import tensorflow as tf
import numpy as np



FLAGS = None

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--foldername', type=str, default='./data-large/')
    parser.add_argument('--batch_size', type=int, default=100)
    parser.add_argument('--num_epoches', type=int, default=100)
    parser.add_argument('--learning_rate', type=float, default=0.5)

    FLAGS = parser.parse_args()
    net = Net(FLAGS.batch_size, FLAGS.learning_rate)

    with tf.Graph().as_default():
        # Dataset is a class for encapsulate the input pipeline
        dataset = Dataset(foldername=FLAGS.foldername,
                              batch_size=FLAGS.batch_size,
                              num_epoches=FLAGS.num_epoches)

        images, labels = dataset.samples_train

        ## The following code defines the network and train
        with tf.device('/gpu:0'): # <==== THIS LINE
            logits = net.inference(images)
            loss = net.loss(logits, labels)
            train_op = net.training(loss)

            init_op = tf.group(tf.initialize_all_variables(), tf.initialize_local_variables())
            sess = tf.Session()
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            start_time = time.time()
            try:
                step = 0
                while not coord.should_stop():
                    _, loss_value = sess.run([train_op, loss])
                    step = step + 1
                    if step % 100 == 0:
                        format_str = ('step %d, loss = %.2f, time: %.2f seconds')
                        print(format_str % (step, loss_value, (time.time() - start_time)))
                        start_time = time.time()
            except tf.errors.OutOfRangeError:
                print('done')
            finally:
                coord.request_stop()

            coord.join(threads)
            sess.close()

usage: ipykernel_launcher.py [-h] [--foldername FOLDERNAME]
                             [--batch_size BATCH_SIZE]
                             [--num_epoches NUM_EPOCHES]
                             [--learning_rate LEARNING_RATE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\singh\AppData\Roaming\jupyter\runtime\kernel-7884e5a0-81c8-4cfd-9bf6-f1c01eed6c45.json


SystemExit: 2

In [86]:
import os
a=os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [87]:
a

'1'

In [111]:
import os
import keras.backend.tensorflow_backend as K
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

config = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth
sess = tf.compat.v1.keras.backend.get_session()

def _get_available_devices():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print (_get_available_devices())

['/device:CPU:0', '/device:GPU:0']
